# How to Fine-Tune LLMs with LoRA Adapters using Hugging Face TRL

This notebook demonstrates how to efficiently fine-tune large language models using LoRA (Low-Rank Adaptation) adapters. LoRA is a parameter-efficient fine-tuning technique that:
- Freezes the pre-trained model weights
- Adds small trainable rank decomposition matrices to attention layers
- Typically reduces trainable parameters by ~90%
- Maintains model performance while being memory efficient

We'll cover:
1. Setup development environment and LoRA configuration
2. Create and prepare the dataset for adapter training
3. Fine-tune using `trl` and `SFTTrainer` with LoRA adapters
4. Test the model and merge adapters (optional)


## 1. Setup development environment

Our first step is to install Hugging Face Libraries and Pytorch, including trl, transformers and datasets. If you haven't heard of trl yet, don't worry. It is a new library on top of transformers and datasets, which makes it easier to fine-tune, rlhf, align open LLMs.


In [ ]:
# Authenticate to Hugging Face
from huggingface_hub import login

# login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

## 2. Load the dataset

In [ ]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Load a sample dataset
from datasets import load_dataset

train_dataset, test_dataset = load_dataset(
    "HuggingFaceH4/ultrachat_200k", split=["train_sft", "test_sft"]
)
column_names = list(train_dataset.features)

print(train_dataset, "\n", test_dataset)

## 3. Fine-tune LLM using `trl` and the `SFTTrainer` with LoRA

The [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `trl` provides integration with LoRA adapters through the [PEFT](https://huggingface.co/docs/peft/en/index) library. Key advantages of this setup include:

1. **Memory Efficiency**: 
   - Only adapter parameters are stored in GPU memory
   - Base model weights remain frozen and can be loaded in lower precision
   - Enables fine-tuning of large models on consumer GPUs

2. **Training Features**:
   - Native PEFT/LoRA integration with minimal setup
   - Support for QLoRA (Quantized LoRA) for even better memory efficiency

3. **Adapter Management**:
   - Adapter weight saving during checkpoints
   - Features to merge adapters back into base model

We'll use LoRA in our example, which combines LoRA with 4-bit quantization to further reduce memory usage without sacrificing performance. The setup requires just a few configuration steps:
1. Define the LoRA configuration (rank, alpha, dropout)
2. Create the SFTTrainer with PEFT config
3. Train and save the adapter weights


In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
checkpoint_path = "microsoft/Phi-4-mini-instruct"
model_kwargs = dict(
    use_cache=False,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",  # loading the model with flash-attention support
    torch_dtype=torch.bfloat16,
    device_map=None,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    ),
)

model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    **model_kwargs
).to(device)

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
tokenizer.model_max_length = 2048
tokenizer.pad_token = (
    tokenizer.unk_token
)  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = "right"

def apply_chat_template(
    example,
    tokenizer,
):
    messages = example["messages"]
    example["text"] = tokenizer.apply_chat_template(
        messages, add_generation_prompt=False, tokenize=False
    )
    return example

# Apply the chat template train dataset
processed_train_dataset = train_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to train_sft",
)

# Apply the chat template test dataset
processed_test_dataset = test_dataset.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    num_proc=10,
    remove_columns=column_names,
    desc="Applying chat template to test_sft",
)

The `SFTTrainer`  supports a native integration with `peft`, which makes it super easy to efficiently tune LLMs using, e.g. LoRA. We only need to create our `LoraConfig` and provide it to the trainer.


In [ ]:
from peft import LoraConfig

# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 16
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 32
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
    modules_to_save=None,
)

Before we can start our training we need to define the hyperparameters (`TrainingArguments`) we want to use.

In [ ]:
# Training configuration
# Hyperparameters based on QLoRA paper recommendations
args = SFTConfig(
    max_steps=40, #-1
    # Output settings
    output_dir="./temp/checkpoint_dir",  # Directory to save model checkpoints
    overwrite_output_dir=True,
    # Training duration
    num_train_epochs=1,  # Number of training epochs
    # Batch size settings
    auto_find_batch_size=True,
    per_device_eval_batch_size=4,
    per_device_train_batch_size=4,  # Batch size per GPU
    remove_unused_columns=True,
    save_steps=100,
    save_total_limit=1,
    seed=0,
    gradient_accumulation_steps=1,  # Accumulate gradients for larger effective batch
    # Memory optimization
    gradient_checkpointing=True,  # Trade compute for memory savings
    gradient_checkpointing_kwargs={"use_reentrant": False},
    # Optimizer settings
    optim="adamw_torch_fused",  # Use fused AdamW for efficiency
    learning_rate=5.0e-06,  # Learning rate (QLoRA paper)
    log_level="info",
    max_grad_norm=0.3,  # Gradient clipping threshold
    # Learning rate schedule
    warmup_ratio=0.2,  # Portion of steps for warmup
    lr_scheduler_type="cosine",  # Keep learning rate constant after warmup
    # Logging and saving
    logging_steps=20,  # Log metrics every N steps
    logging_strategy="steps",
    save_strategy="epoch",  # Save checkpoint every epoch
    # Precision settings
    bf16=True,  # Use bfloat16 precision
    do_eval=False,
    # Integration settings
    push_to_hub=False,  # Don't push to HuggingFace Hub
    report_to="none",  # Disable external logging
    eval_accumulation_steps=1,
)

We now have every building block we need to create our `SFTTrainer` to start then training our model.

In [ ]:
# Create SFTTrainer with LoRA configuration
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_test_dataset,
    peft_config=peft_config,  # LoRA configuration
    processing_class=tokenizer,
)

Start training our model by calling the `train()` method on our `Trainer` instance. This will start the training loop and train our model for 3 epochs. Since we are using a PEFT method, we will only save the adapted model weights and not the full model.

In [ ]:
# start training, the model will be automatically saved to the hub and the output directory
# train_result = trainer.train()
trainer.train()

# metrics = train_result.metrics
# trainer.log_metrics("train", metrics)
# trainer.save_metrics("train", metrics)
# trainer.save_state()

# evaluate training
# tokenizer.padding_side = "left"
# metrics = trainer.evaluate()
# metrics["eval_samples"] = len(processed_test_dataset)
# trainer.log_metrics("eval", metrics)
# trainer.save_metrics("eval", metrics)

# save model
trainer.save_model(args.output_dir)

The training with Flash Attention for 3 epochs with a dataset of 15k samples took 4:14:36 on a `g5.2xlarge`. The instance costs `1.21$/h` which brings us to a total cost of only ~`5.3$`.



### Merge LoRA Adapter into the Original Model

When using LoRA, we only train adapter weights while keeping the base model frozen. During training, we save only these lightweight adapter weights (~2-10MB) rather than a full model copy. However, for deployment, you might want to merge the adapters back into the base model for:

1. **Simplified Deployment**: Single model file instead of base model + adapters
2. **Inference Speed**: No adapter computation overhead
3. **Framework Compatibility**: Better compatibility with serving frameworks


In [ ]:
from peft import AutoPeftModelForCausalLM
import gc

# free the memory
gc.collect()
torch.cuda.empty_cache()

# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(
    args.output_dir, safe_serialization=True, max_shard_size="2GB"
)

## 3. Test Model and run Inference

After the training is done we want to test our model. We will load different samples from the original dataset and evaluate the model on those samples, using a simple loop and accuracy as our metric.



In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

gc.collect()
torch.cuda.empty_cache()

# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir, device_map="auto", torch_dtype=torch.float16
)

Lets test some prompt samples and see how the model performs.

In [ ]:
def response(prompt):
    tokenized_chat = tokenizer.apply_chat_template(
        [{"role": "user", "content": prompt}],
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    )
    outputs = model.generate(tokenized_chat, max_new_tokens=128)
    return tokenizer.decode(outputs[0])

prompts = [
    "What is the capital of Germany? Explain why thats the case and if it was different in the past?",
    "Write a Python function to calculate the factorial of a number.",
    "A rectangular garden has a length of 25 feet and a width of 15 feet. If you want to build a fence around the entire garden, how many feet of fencing will you need?",
    "What is the difference between a fruit and a vegetable? Give examples of each.",
]

for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{response(prompt)}")
    print("-" * 50)